# Frequency

In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd

feature_pos = [[0,8],[8,12],[12,28],[28,44],[44,52],[52,64],[64,80],[80,92],[92,102]] 
df_train = pd.read_csv('train_kaggle.csv')
df_test = pd.read_csv('sample_solution.csv')
Y = df_train['Label'].values

## Save Frequency Features
Skip this if you have saved the features

In [ ]:
import numpy as np

dataframes = np.load('allData.npy', allow_pickle = True)
dataframes.shape

def load_test_dataframe(id):
    test_data = np.load("test/test/{}.npy".format(id))
    return pd.DataFrame(data=test_data)

testdatas = []
for id in df_test['Id']:
    dfi = load_test_dataframe(id)
    testdatas.append(dfi.values)

In [ ]:
feat = dataframes[0].shape[1]
# feat = 12

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

def one_hot_encoding(encoder, dfs, col):
    if encoder == None:
        data = set(dfs[0][:,col])
        for i in range(1,len(dfs)):
            df = dfs[i]
            data |= set(df[:,col])
        encoder = OneHotEncoder(handle_unknown='ignore')
        encoder = encoder.fit([[str(e)] for e in data])
    encodedData = []
    for df in dfs:
        mat = np.sum(encoder.transform([[str(e)] for e in df[:,col]]), axis=0)
        mat = np.squeeze(np.asarray(mat))
        encodedData.append(mat)
    encodedData = np.stack(encodedData)    
    return encoder, encodedData

def one_hot_encoding_feature(encoders, dfs, feature):
    if len(encoders) == 0:
        encoder, data = one_hot_encoding(None, dfs, feature_pos[feature][0])
        encoders.append(encoder)    
        for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
            encoder, dataCol = one_hot_encoding(None, dfs, col)
            encoders.append(encoder)
            data = np.concatenate((data, dataCol), axis=1)
    else:
        encoder, data = one_hot_encoding(encoders[0], testdatas, feature_pos[feature][0])
        for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
            i = col - feature_pos[feature][0]
            encoder, dataCol = one_hot_encoding(encoders[i], testdatas, col)
            data = np.concatenate((data, dataCol), axis=1)
    return encoders, data

In [ ]:
import gc

# Give up last feature
for feat in range(len(feature_pos) - 1):
    print(feat)
    encoders, train = one_hot_encoding_feature([], dataframes, feat)
    encoders, test = one_hot_encoding_feature(encoders, testdatas, feat)
    np.save('freq/' + str(feat) + '_train.npy', train)
    np.save('freq/' + str(feat) + '_test.npy', test)
    gc.collect()

## Load Frequency Features

In [5]:
import numpy as np
# last feature must be false
feature_selected = [True,True,True,True,True,True,True,True,False] 

def load_freq(feat, postfix):
    data = np.load("freq/{}_{}.npy".format(feat, postfix))
    return data

def load_test_freq(feat):
    return load_freq(feat, 'test')

def load_train_freq(feat):
    return load_freq(feat, 'train')

In [6]:
from tqdm import tqdm

XTrain = []
XTest = []
for feat in tqdm(range(len(feature_pos))):
    if feature_selected[feat] == False:
        continue
    train = load_train_freq(feat)
    test = load_test_freq(feat)
    XTrain.append(train)
    XTest.append(test)
XTrain = np.concatenate(XTrain, axis=1)
XTest = np.concatenate(XTest, axis=1)

100%|████████████████████████████████████████████████████████████| 9/9 [00:34<00:00,  3.82s/it]


## Train models

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(XTrain, Y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(14929, 69548)
(3733, 69548)
(14929,)
(3733,)


In [8]:
X_train

array([[ 34.,   3.,   0., ...,   0.,   0.,   0.],
       [ 44.,  11.,   0., ...,   0.,   0.,   0.],
       [119.,   6.,   0., ...,   0.,   0.,   0.],
       ...,
       [ 32.,   3.,   0., ...,   0.,   0.,   0.],
       [ 34.,   3.,   0., ...,   0.,   0.,   0.],
       [ 32.,   3.,   0., ...,   0.,   0.,   0.]])

In [9]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train.astype(np.int32), y_train.astype(int))
valid_set = lgb.Dataset(X_valid.astype(np.int32), y_valid.astype(int))

from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(XTrain, Y)
modelSelection = SelectFromModel(clf, prefit=True, max_features=8000)
XTrain = modelSelection.transform(XTrain)
XTrain.shape

XTest = modelSelection.transform(XTest)
XTest.shape

In [10]:
from hyperopt import hp
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import tpe
import lightgbm as lgb
from hyperopt import STATUS_OK

ITER = 50
STOP_ROUND = 5

# Create the dataset
def objective(params):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    params['num_leaves'] = int(params['num_leaves'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    bst = lgb.train(params, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
    bst.save_model('model.txt', num_iteration=bst.best_iteration)
  
    # Extract the best score
    best_score = bst.best_score['valid_0']['auc']
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

# Define the search space
space = {
    'boosting_type': hp.choice('boosting_type', 
                                [
                                    'gbdt',
                                    'dart',
                                    'goss'
                                ]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'metric': 'auc'
}

# Trials object to track progress
bayes_trials = Trials()

MAX_EVALS = 500

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials)
best

ModuleNotFoundError: No module named 'hyperopt'

In [ ]:
best['boosting_type'] = 'dart'
best['num_leaves'] = int(best['num_leaves'])
best['subsample_for_bin'] = int(best['subsample_for_bin'])
best['min_child_samples'] = int(best['min_child_samples'])
lgbModel = lgb.train(best, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
lgbModel.save_model('model.txt', num_iteration=bst.best_iteration)
YTest = lgbModel.predict(XTest)

In [ ]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(boosting_type=best['boosting_type'],
                        num_leaves=best['num_leaves'],
                        learning_rate=best['learning_rate'],
                        subsample_for_bin=best['subsample_for_bin'],
                        min_child_samples=best['min_child_samples'],
                        reg_alpha=best['reg_alpha'],
                        reg_lambda=best['reg_lambda'],
                        colsample_bytree=best['colsample_by_tree'])
model.fit(XTrain, Y)
probs = model.predict_proba(XTest, num_iteration=lgbModel.best_iteration)

In [12]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(XTrain, Y)
probs = model.predict_proba(XTest)

In [13]:
YTest = probs[:, 0]

In [14]:
df_test['Predicted'] = YTest
df_test

,Id,Predicted
0,0,0.001261
1,1,0.007353
2,2,0.998756
3,3,0.014063
4,4,0.508659
5,5,0.903903
6,6,0.999023
7,7,0.916970
8,8,0.001128
9,9,0.000531


In [15]:
df_test.to_csv('test.csv', index=False)